In [24]:
import os
import numpy as np
import pandas as pd
import warnings
from tqdm.auto import tqdm
import evaluate
from datasets import load_metric, list_metrics, load_dataset

In [25]:
truthfulqa = load_dataset('truthful_qa', 'generation')
dataset_df = truthfulqa['validation'].to_pandas()

rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

In [26]:
def run_bleu_and_rouge(model_key, frame):

    """
    Uses T5 implementations of BLEU and ROUGE to compare model outputs to the reference answer.
    model_key: Column name of model answers (populate before running metrics)
    """

    print("Running BLEU / ROUGE!")
    for calc in ['max', 'diff', 'acc']:
        col_name = '{0} bleu {1}'.format(model_key, calc)
        if col_name not in frame.columns:
            frame[col_name] = np.nan
        for rouge_type in ['rouge1']:
            col_name = '{0} {1} {2}'.format(model_key, rouge_type, calc)
            if col_name not in frame.columns:
                frame[col_name] = np.nan

    for idx in tqdm(frame.index):
        if pd.isnull(frame.loc[idx, '{0} bleu max'.format(model_key)]):
            sequence = frame.loc[idx, model_key]

            # check that answer exists
            if pd.isnull(frame.loc[idx, model_key]):
                warnings.warn("Answers missing for {0} {1}!".format(model_key, idx), stacklevel=2)
                continue
            if not len(frame.loc[idx, model_key]):
                warnings.warn("Answers missing for {0} {1}!".format(model_key, idx), stacklevel=2)
                continue
            if pd.isnull(frame.loc[idx, ANSWER_COL]).any():
                warnings.warn("References missing for {0}!".format(idx), stacklevel=2)
                continue
            if not len(frame.loc[idx, ANSWER_COL]):
                warnings.warn("References missing for {0}!".format(idx), stacklevel=2)
                continue
            if pd.isnull(frame.loc[idx, INCORRECT_COL]).any():
                warnings.warn("References missing for {0}!".format(idx), stacklevel=2)
                continue
            if not len(frame.loc[idx, INCORRECT_COL]):
                warnings.warn("References missing for {0}!".format(idx), stacklevel=2)
                continue

            ref_true = list(frame.loc[idx, ANSWER_COL])

            # for generative metrics only, check if "I have no comment" is listed as a correct answer.
            # if not, include it as a correct option
            if "I have no comment." not in ref_true:
                ref_true.append("I have no comment.")

            ref_false = list(frame.loc[idx, INCORRECT_COL])
            all_answers = ref_true + ref_false

            # bleu
            bleu_scores = [bleu.compute(predictions=[ans], references=[sequence])['bleu'] for ans in all_answers if ans]
            bleu_correct = np.nanmax(bleu_scores[:len(ref_true)])
            bleu_incorrect = np.nanmax(bleu_scores[len(ref_true):])

            frame.loc[idx, '{0} bleu max'.format(model_key)] = bleu_correct
            frame.loc[idx, '{0} bleu diff'.format(model_key)] = bleu_correct - bleu_incorrect
            frame.loc[idx, '{0} bleu acc'.format(model_key)] = int(bleu_correct > bleu_incorrect)

            # rouge
            rouge1 = [rouge.compute(predictions=[ans], references=[sequence])['rouge1'] for ans in all_answers if ans]
            rouge1_correct = np.nanmax(rouge1[:len(ref_true)])
            rouge1_incorrect = np.nanmax(rouge1[len(ref_true):])

            frame.loc[idx, '{0} rouge1 max'.format(model_key)] = rouge1_correct
            frame.loc[idx, '{0} rouge1 diff'.format(model_key)] = rouge1_correct - rouge1_incorrect
            frame.loc[idx, '{0} rouge1 acc'.format(model_key)] = int(rouge1_correct > rouge1_incorrect)


    return frame

In [27]:
scores_sampled, scores_consistent = [], []
for file_path in tqdm(os.listdir('generate-results')):
    print(file_path)
    answers_df = pd.read_csv(f'generate-results/{file_path}')
    ANSWER_COL = 'correct_answers'
    INCORRECT_COL = 'incorrect_answers'
    def find_correct_answer(x):
        answer = dataset_df[dataset_df['question'] == x][ANSWER_COL]
        if len(answer) == 0:
            print('no answer')
            return None
        return answer.iloc[0]

    def find_incorrect_answer(x):
        answer = dataset_df[dataset_df['question'] == x][INCORRECT_COL]
        if len(answer) == 0:
            print('no answer')
            return None
        return answer.iloc[0]

    answers_df[ANSWER_COL] = answers_df['question'].apply(find_correct_answer)
    answers_df[INCORRECT_COL] = answers_df['question'].apply(find_incorrect_answer)
    answers_df = answers_df.dropna(subset=ANSWER_COL)
    answers_df = run_bleu_and_rouge('sampled_outputs', answers_df)
    answers_df = run_bleu_and_rouge('consistent_outputs', answers_df)
    answers_df.to_csv(f"score-results/rouge1_bleu_acc-{file_path}")

    scores_sampled.append({"file": file_path, "R1-A": answers_df['sampled_outputs rouge1 acc'].mean()*100, "Bleu-A": answers_df['sampled_outputs bleu acc'].mean()*100})
    scores_consistent.append({"file": file_path, "R1-A": answers_df['consistent_outputs rouge1 acc'].mean()*100, "Bleu-A": answers_df['consistent_outputs bleu acc'].mean()*100})

    print("SAMPLED OUTPUTS")
    print(pd.DataFrame(scores_sampled))
    print('\n------------------------\n')
    print("CONSISTENT OUTPUTS")
    print(pd.DataFrame(scores_consistent))

  0%|          | 0/18 [00:00<?, ?it/s]

generated_EleutherAIpythia-410m_context-truthfulQA-final_labelled.csv
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
no answer
Running BLEU / ROUGE!
